In [56]:
# Import stanza
import stanza

# Download the Stanford CoreNLP package with Stanza's installation command
# This'll take several minutes, depending on the network speed
corenlp_dir = './corenlp'
#stanza.install_corenlp(dir=corenlp_dir)

# Set the CORENLP_HOME environment variable to point to the installation location
import os
os.environ["CORENLP_HOME"] = corenlp_dir

# Import client module
from stanza.server import CoreNLPClient

# Import tqdm for loading 
from tqdm import tqdm

# Import pandas for Dataframe
import pandas as pd

# Setting AllenNLP
from allennlp.predictors.predictor import Predictor
from allennlp_models.coref.predictors.coref import CorefPredictor
from allennlp_models.coref.dataset_readers.conll import ConllCorefReader
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2020.02.27.tar.gz")

# Import Text
text = "Huawei Technologies Co., Ltd.is a Chinese multinational technology company headquartered in Shenzhen, Guangdong. It designs, develops, and sells telecommunications equipment and consumer electronics (mainly smartphones). The company was founded in 1987 by Ren Zhengfei, a former Deputy Regimental Chief in the People's Liberation Army. Huawei has deployed its products and services in more than 170 countries. It overtook Ericsson in 2012 as the largest telecommunications equipment manufacturer in the world, and overtook Apple in 2018 as the second-largest manufacturer of smartphones in the world, behind Samsung Electronics. In December 2019, Huawei reported that its annual revenue had risen to $121.72 billion US dollars in 2019. Although successful internationally, Huawei has faced difficulties in some markets, due to claims of undue state support, links to the People's Liberation Army, and cybersecurity concerns—primarily from the United States government—that Huawei's infrastructure equipment may enable surveillance by the Chinese government. With the development of 5G wireless networks, there have been calls from the U.S. and its allies to not do any kind of business with Huawei or other Chinese telecommunications companies such as ZTE. Huawei has argued that its products posed no greater cybersecurity risk than those of any other vendor and that there is no evidence of the U.S. espionage claims. Questions regarding Huawei's ownership and control as well as concerns regarding the extent of state support also remain. Huawei has also been accused of assisting in the surveillance and mass detention of Uyghurs in the Xinjiang re-education camps, which have resulted in sanctions by the United States Department of State. In the midst of an ongoing trade war between China and the United States, Huawei was restricted from doing commerce with U.S. companies due to alleged previous willful violations of U.S. sanctions against Iran. On 29 June 2019, U.S. President Donald Trump reached an agreement to resume trade talks with China and announced that he would ease the aforementioned sanctions on Huawei. Huawei cut 600 jobs at its Santa Clara research center in June, and in December 2019 founder Ren Zhengfei said it was moving the center to Canada because the restrictions would block them from interacting with US employees. In July 2020, Huawei surpassed Samsung and Apple to become the leading smartphone mobile brand in the world for the first time primarily due to a drop in Samsung's global sales in the second quarter of 2020 owing to the impact of the COVID-19 pandemic."

# View Coref Clusters and their positions
tks = predictor.predict(text)['document']
cluster = predictor.predict(text)['clusters']
coref_positions = []
corrected_coref_positions = []
start = 0
import re

print('Extracting Coreference Clusters and Positions')
for i in tqdm(range(len(cluster))):
    coref_position = []
    for j in range(len(cluster[i])):
        coref = ' '.join(tks[cluster[i][j][0]:cluster[i][j][1] + 1])
        #coref = re.sub("\s*(\W)\s*",r"\1",coref)
        coref = re.sub(" , ", r", ", coref)
        coref = re.sub(" ,", r", ", coref)
        coref = re.sub(" 's", r"'s", coref)
        coref = re.sub("[(] ", r"(", coref)
        coref = re.sub(" [)]", r")", coref)
        coref = re.sub("[$] ", r"$", coref)
        coref = re.sub("RM   ", r"RM ", coref)
        
        start_index = text.find(coref, start)
        if start_index == -1:
            start_index = text.find(coref)
        end_index   = start_index+len(coref)
        start = end_index + 1
        position = list([coref, start_index, end_index])
        coref_position.append(position)
    coref_positions.append(coref_position)
print('\nDone Extraction!')

pd.DataFrame(coref_positions) 

  0%|          | 0/10 [00:00<?, ?it/s]

Extracting Coreference Clusters and Positions


100%|██████████| 10/10 [00:00<00:00, 154.72it/s]



Done Extraction!


,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,"[Huawei Technologies Co., Ltd.is, 0, 31]","[It, 113, 115]","[The company, 221, 232]","[Huawei, 336, 342]","[its, 356, 359]","[It, 410, 412]","[Huawei, 647, 653]","[its, 668, 671]","[Huawei, 773, 779]","[Huawei's, 973, 981]",...,"[Huawei, 1257, 1263]","[its, 1280, 1283]","[Huawei's, 1440, 1448]","[Huawei, 1542, 1548]","[Huawei, 1819, 1825]","[Huawei, 2120, 2126]","[Huawei, 2128, 2134]","[its, 2151, 2154]","[it, 2239, 2241]","[Huawei, 2366, 2372]"
1,"[the world, 2450, 2459]","[the world, 499, 508]","[the world, 590, 599]",None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,"[the People's Liberation Army, 867, 895]","[the People's Liberation Army, 306, 334]",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,"[the United States government, 939, 967]","[the U.S., 1131, 1139]","[its, 1144, 1147]","[the United States, 1706, 1723]",None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,"[the Chinese government, 1034, 1056]","[China, 1790, 1795]","[China, 2049, 2054]",None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,"[U.S. President Donald Trump, 1973, 2000]","[he, 2005, 2007]",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,"[December 2019, 2199, 2212]","[December 2019, 632, 645]",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
7,"[Ren Zhengfei, a former Deputy Regimental Chie...","[founder Ren Zhengfei, 2213, 2233]",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,"[its Santa Clara research center, 2151, 2182]","[the center, 2253, 2263]",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9,"[Samsung Electronics, 608, 627]","[Samsung, 2383, 2390]","[Samsung's, 2506, 2515]",None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
